<a href="https://colab.research.google.com/github/google-research/protein-ligand-binding-free-energy-calculations/blob/matteo-dev/colab_tutorial/gromacs_abfe_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2022 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Dependencies and Mini Gromacs Tutorial

## Note

1. If you use a public colab instance, remember to change your runtime type to GPU
2. If you deployed a custom GCE runtime, please connect to a GPU VM instance.

## Summary

1. Pull Gromacs 2022.3 binary compiled specifically for Colab GPU environment and install the software.
2. Install Gromacs Python API on the fly, together with other packages used in the tutorial.
3. Run simulations for benchmarks and show how to use the official GMX
Python API to modify inputs.
4. Visualize the output conformation for the longer simulation.

In [8]:
#@title Install Gromacs and Gromacs Python API
!wget https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz -O /tmp/gromacs.tar.gz
!wget https://storage.googleapis.com/gromacs-bin/benchMEM.tpr -O /tmp/benchMEM.tpr
!wget https://storage.googleapis.com/gromacs-bin/hif2a_eq.tpr -O /tmp/hif2a_eq.tpr

!tar zxf /tmp/gromacs.tar.gz
!sudo rm -rf /usr/local/gromacs
!sudo mv gromacs-avx /usr/local/gromacs
!rm /tmp/gromacs.tar.gz

!pip3 install --upgrade pip
!pip3 install setuptools wheel cmake pybind11 py3DMol
# This ensure we always install the latest PMX version from GitHub.
!if [ -d /usr/local/lib/python3.7/dist-packages/pmx/ ]; then yes Y | pip3 uninstall pmx; fi
!pip3 install git+https://github.com/deGrootLab/pmx.git@abfe_dev
!gmxapi_ROOT=/usr/local/gromacs/ pip3 install --no-cache-dir gmxapi

--2022-11-04 18:28:30--  https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 172.253.117.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23920923 (23M) [application/x-gzip]
Saving to: ‘/tmp/gromacs.tar.gz’

/tmp/gromacs.tar.gz 100%[===================>]  22.81M  --.-KB/s    in 0.1s    

2022-11-04 18:28:30 (208 MB/s) - ‘/tmp/gromacs.tar.gz’ saved [23920923/23920923]

--2022-11-04 18:28:30--  https://storage.googleapis.com/gromacs-bin/benchMEM.tpr
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 172.253.117.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3461380 (3.3M) [application/octet-stream]
Saving to:

In [1]:
#@title Set Level for Logging
#@markdown Sets the python logging level. This is useful for debugging.
import logging

log_level = 'INFO' #@param ['DEBUG', 'INFO', 'ERROR']

_log_level = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'ERROR': logging.ERROR
}

logging.basicConfig(level=_log_level[log_level])

In [2]:
#@title [Please Rerun after Restart] Create a Session
#@markdown Please rerun this cell every time when you restart the runtime.
#@markdown A new session ID will be created for a new colab runtime session.
#@markdown Run this cell again will refresh the session ID.

import os
import time

import gmxapi


# Sets a session ID for this session.
_SESSION_ID = str(int(time.time())) #@param

# Monkey patch this session ID into the gmxapi.operation.ResourceManager.
# Hack. Do not use in the user code.
# This is temporarily added to allow users to start from a new set of working
# directories for all the mdrun launches. GMX API doesn't expose the
# underlying resource manager or context yet.
def operation_id(self):
  return f"{self._base_operation_id}_{_SESSION_ID}"
gmxapi.operation.ResourceManager.operation_id = property(operation_id)


def get_work_dir(md_output):
  values = [v for v in md_output.values()]
  return values[0].result()


def tail_log(md_output, output_file_base, last_n_lines=6):
  work_dir = get_work_dir(md_output)

  with open(os.path.join(work_dir, output_file_base + '.log'), 'r') as log:
    for line in log.readlines()[-last_n_lines:]:
      print(line)

print("Session ID: ", _SESSION_ID)

INFO:gmxapi:Importing gmxapi.
INFO:gmxapi.operation:Importing gmxapi.operation
INFO:gmxapi.commandline:Importing gmxapi.commandline
INFO:gmxapi.simulation.workflow:Importing gmx.workflow
INFO:gmxapi.simulation.workflow:Using schema version gmxapi_workspec_0_1.
INFO:gmxapi.mdrun:Importing gmxapi.simulation.mdrun
INFO:gmxapi.read_tpr:Importing gmxapi.simulation.read_tpr
INFO:gmxapi.modify_input:Importing gmxapi.simulation.modify_input


Session ID:  1667596348


In [11]:
#@title Run a Short Simulation for Benchmark

# Read a TPR sample and run it.
# The TPR file configures a 20ps simulation.
input_tpr = gmxapi.read_tpr('/tmp/benchMEM.tpr')

# Can specify computations to GPU.
output_file_base = 'short'
md = gmxapi.mdrun(input_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})

# Run the simulation
md.run()

# Tail performance
tail_log(md.output, output_file_base)

INFO:gmxapi.mdrun:Building mdrun operation from source <gmxapi.simulation.read_tpr.StandardOperationHandle object at 0x7f156fb49710>
INFO:gmxapi.mdrun:mdrun receiving input _simulation_input: ResultDescription(dtype=<class 'str'>, width=1)
INFO:gmxapi.mdrun:mdrun input bound as source collection DataSourceCollection([('_simulation_input', <Future: name='_simulation_input', description=ResultDescription(dtype=<class 'str'>, width=1)>), ('parameters', <Future: name='parameters', description=ResultDescription(dtype=<class 'dict'>, width=1)>), ('runtime_args', {'-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu', '-deffnm': 'short'})])
INFO:gmxapi.simulation.context:Importing gmxapi.simulation.context
INFO:gmxapi.simulation.context:mpi4py is not available for default session communication.
INFO:gmxapi.mdrun:Created /content/mdrun_0_1667586554_0/topol.tpr on rank 0
INFO:gmxapi.simulation.workflow:Added element md_sim to workspec.
INFO:gmxapi.simulation.context:Collected builder 

               Core t (s)   Wall t (s)        (%)

       Time:      168.602       42.151      400.0

                 (ns/day)    (hour/ns)

Performance:       41.000        0.585

Finished mdrun on rank 0 Fri Nov  4 18:30:00 2022





In [ ]:
#@title Modified Params and Run A Longer Simulation
# Run the simulation for 40 ps.
output_file_base = 'long2'

modified_tpr = gmxapi.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})

md = gmxapi.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

In [ ]:
#@title What about a binding affinity simulation with HIF2a?
output_file_base = 'hif'

input_tpr = gmx.read_tpr('/tmp/hif2a_eq.tpr')
modified_tpr = gmx.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})
md = gmx.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

## Visualization

Visualize the output from the above HIF2A simulation.
We use py3DMol to visualize the output gro file.

In [ ]:
#@title Visualize the output conformation of the HIF2A simulation.
import py3Dmol

view = py3Dmol.view()
work_dir = get_work_dir(md.output)

view.addModel(
    open(os.path.join(work_dir, output_file_base + '.gro'), 'r').read(), 'gro')

view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({}, {'cartoon': {'color': 'spectrum'}})

view.show()

# ABFE Workflow
-----

In [45]:
import shutil
import numpy as np
from tqdm import tqdm
import pmx
from pmx.AbsoluteDG import AbsoluteDG

In [67]:
#@title Define some helper functions

def mdrun(tpr, pmegpu=True, nsteps=None):
  """Wrapper for gmxapi.mdrun with predefined scenarios optimized for different 
  types of simulations.

  Args:
    tpr (str): path to TPR file.
    pmegpu (bool): whether to run PME calculations on the GPU. Default is True.
      Note this is not possible with certain integrators.
    nsteps (int): number of integration steps to run, overwrites nsteps defined
      by the MDP file used to generate the TPR. Default is None (do not modify 
      TPR).

  Returns:
    object: StandardOperationHandle returned by gmxapi.mdrun.
  """
  # Load TPR file.
  input_tpr = gmxapi.read_tpr(tpr)
  if nsteps is not None:
    input_tpr = gmxapi.modify_input(input=input_tpr, 
                                    parameters={'nsteps': nsteps})
    

  # Use deffnm with full path.
  #_deffnm = "/".join(tpr.split('/')[:-1]) + f"/{deffnm}"
  path = "/".join(tpr.split('/')[:-1])
  
  # Specify command line flags and arguments to be passed to mdrun.
  # Note: energy minimization cannot run PME on GPU.
  if pmegpu:
    _pme = 'gpu'
    _pmefft = 'gpu'
    _bonded = 'gpu'
  else:
    _pme = 'auto'
    _pmefft = 'auto'
    _bonded = 'auto'

  md = gmxapi.mdrun(input_tpr,
                    runtime_args={'-nb': 'gpu', 
                                  '-pme': _pme, 
                                  '-pmefft': _pmefft, 
                                  '-bonded': _bonded,
                                  '-x': f'{path}/traj.xtc',
                                  '-o': f'{path}/traj.trr',
                                  '-c': f'{path}/confout.gro',
                                  '-e': f'{path}/ener.edr', 
                                  '-g': f'{path}/md.log',
                                  '-cpo': f'{path}/state.cpt'
                                  }
                    )

  # Run the simulation.
  md.run()
  return md


def mdrun_completed(tpr: str) -> bool:
  """Checks whether energy minimization completed successfully.
  
  Args:
    tpr (str): path to TPR file.

  Returns:
    bool: whether the TPR has been run successfully.
  """
  # (maldeghi): IIRC Gromacs would output the GRO file only for mdruns that
  # did not crash/errored. We can make this stricted by checking the log
  # file too.
  gro = tpr.replace('tpr.tpr', 'confout.gro')
  if os.path.isfile(gro):
    return True
  else:
    return False

In [60]:
#@title Copy input files

_WORKDIR = f"/content/pmxrun_{_SESSION_ID}"

if not os.path.isdir(_WORKDIR):
  os.mkdir(_WORKDIR)

  # Copy topology and structure files for protein and ligand.
  shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/struct_top/', f'{_WORKDIR}/struct_top/')

  # Copy MDP files (i.e. Gromacs config files).
  shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/mdppath/', f'{_WORKDIR}/mdppath/')

# Quick fix to the MDP files: use pme-order 4 becuase this is the only order
# supported to be able to run PME on the GPU.
! sed -i 's/pme-order.*/pme-order = 4/g' $_WORKDIR/mdppath/*

# Show contents of our working directory.
!ls -l $_WORKDIR

total 12
drwxr-xr-x 3 root root 4096 Nov  4 21:12 lysopath
drwxr-sr-x 2 root root 4096 Nov  4 21:54 mdppath
drwxr-sr-x 4 root root 4096 Nov  4 18:29 struct_top


## 1.&nbsp; Input Files Preparation

In [6]:
#@title 1.1&nbsp; Setup folder structure

# Initialize the free energy environment object. It will store the main 
# parameters for the calculations.
fe = AbsoluteDG(ligList=['lysozyme_benzene'], 
                apoCase='lysozyme_apo', 
                bDSSB=False,
                gmxexec='/usr/local/gromacs/bin/gmx')

# Set the workpath in which simulation input files will be created.
fe.workPath = f'{_WORKDIR}/lysopath'
# Set the path to the MDP files.
fe.mdpPath = f'{_WORKDIR}/mdppath'
# Set the number of replicas.
fe.replicas = 3
# Provide the path to the structures and topologies.
fe.structTopPath = f'{_WORKDIR}/struct_top'

# Prepare the directory structure with all simulations steps required.
fe.simTypes = ['em_posre',  # Energy minimization.
               'eq_posre',  # Equilibrium sim with position restraints.
               'eq',  # Equilibrium simulation.
               'transitions']  # Alchemical, non-equilibrium simulations.

fe.prepareFreeEnergyDir()


---------------------
Summary of the setup:
---------------------

   workpath: /content/pmxrun_1667596348/lysopath
   mdp path: /content/pmxrun_1667596348/mdppath
   # ligands: 1
   ligands:
        lysozyme_benzene
   apo state: lysozyme_apo

---------------------
Directory structure:
---------------------

/content/pmxrun_1667596348/lysopath/
|
|--ligX
|--|--water
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--protein
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--strTopFolder
|--lig..

DONE


In [7]:
#@title 1.2&nbsp; Assemble simulation systems

# Assemble the systems: build Gromacs structure and topology for the 
# ligand+water and ligand+protein+water systems.
fe.assemble_systems()

# Define the simulation boxes, fill them with water molecules, and add ions to 
# neutralize the system and reach desired NaCl concentration (0.15 M by default).
fe.boxWaterIons()

----------------------
Assembling the systems
----------------------
Order: ligand-protein-other-water
--- Assembling structures: lysozyme_benzene ---
--- Assembling topologies: lysozyme_benzene ---
----------------
Box, water, ions
----------------


In [8]:
# Check the files present now. This is the content of root dir of the 
# calculation for this protein-ligand pair. The following ls calls show 
# the content of some subfolders for the complex simulations, 
# stateA (coupled ligand). The other folders have the same structure/files but 
# for the ligand in water simulations and for stateB too.
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/"
!ls $_WORKDIR/lysopath/lysozyme_benzene/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein/stateA"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/run1/"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/run1/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre/

> ls $/content/pmxrun_1667596348/lysopath/lysozyme_benzene/
protein  strTopFolder_apo  strTopFolder_holo  water

> ls $/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein
stateA	stateB

> ls $/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA
box.pdb   mdout.mdp  run2  system.pdb  top.top	water.pdb
ions.pdb  run1	     run3  topol.top   tpr.tpr

> ls $/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run1/
em_posre  eq  eq_posre	transitions

> ls $/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre


## 2&nbsp; Energy Minimization

In [62]:
#@title 2.1&nbsp; Prepare TPR files

# Call grompp and create TPR files for all systems and repeats.
# (maldeghi): why EM with -DPOSRES? To be checked.
tpr_files = fe.prepare_simulation(simType='em_posre')
print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

# eq_posre
#fe.prepare_simulation( simType='eq_posre', prevSim='em_posre' )

# eq
#fe.prepare_simulation( simType='eq', prevSim='eq_posre' )

# transitions
#fe.equilTime = 1080.0 # ps to discard as equilibration
#fe.bGenTiTpr = True

# analysis
#fe.run_analysis(ligs=['lysozyme_benzene'] )
#fe.analysis_summary()

-----------------------------------------
Preparing simulation: em_posre
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateA/run1/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateA/run2/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateA/run3/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateB/run1/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateB/run2/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateB/run3/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run2/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run3/em_posre/tpr.tpr
  /content/pmxrun_1667596348/lysopa

In [68]:
#@title 2.2&nbsp; Run minimizations

# Read the TPR files and run all minimizations.
for tpr_file in tqdm(tpr_files):

  # If minimization has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=False, nsteps=1000)

  0%|          | 0/12 [00:00<?, ?it/s]INFO:gmxapi.modify_input:Building mdrun operation from source <gmxapi.simulation.read_tpr.StandardOperationHandle object at 0x7f99ec1f3910>
INFO:gmxapi.modify_input:modify_input receiving input _simulation_input: ResultDescription(dtype=<class 'str'>, width=1)
INFO:gmxapi.modify_input:modify_input input bound as source collection DataSourceCollection([('_simulation_input', <Future: name='_simulation_input', description=ResultDescription(dtype=<class 'str'>, width=1)>), ('parameters', {'nsteps': 1000})])
INFO:gmxapi.mdrun:Building mdrun operation from source <gmxapi.simulation.modify_input.StandardOperationHandle object at 0x7f99ecc22150>
INFO:gmxapi.mdrun:mdrun receiving input _simulation_input: ResultDescription(dtype=<class 'str'>, width=1)
INFO:gmxapi.mdrun:mdrun input bound as source collection DataSourceCollection([('_simulation_input', <Future: name='_simulation_input', description=ResultDescription(dtype=<class 'str'>, width=1)>), ('paramete

`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateA/run1/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateA/run2/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateA/run3/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateB/run1/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateB/run2/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/water/stateB/run3/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run2/em_posre/tpr.tpr` already ran successfully
`/content/pmxrun_1667596348/lysopath/lysozyme_benzene/protein/stateA/run3/em

INFO:gmxapi.simulation.context:Collected builder for md_sim
INFO:gmxapi.simulation.context:Subscribing md_sim to tpr_input.
INFO:gmxapi.simulation.context:Building sequence ['tpr_input', 'md_sim']
INFO:gmxapi.simulation.context:Building <gmxapi.simulation.context._load_tpr.<locals>.Builder object at 0x7f99ec1cb990>
INFO:gmxapi.simulation.context:Building <gmxapi.simulation.context._md.<locals>.Builder object at 0x7f99ec1cbd10>
INFO:gmxapi.simulation.context:Launching work on context rank 0, subcommunicator rank 0.
INFO:gmxapi.simulation.context:rank 0 changed directory to /content/mdrun_24_1667596348_0/mdrun_25_1667596348_0/mdrun_26_1667596348_0/mdrun_27_1667596348_0
INFO:gmxapi.simulation.context:Loading TPR file: /content/mdrun_24_1667596348_0/mdrun_25_1667596348_0/mdrun_26_1667596348_0/mdrun_27_1667596348_0/topol.tpr
INFO:gmxapi.simulation.context:Exiting session on context rank 0.
INFO:gmxapi.simulation.context:Calling session.close().
INFO:gmxapi.simulation.context:Freeing sub-com

## 3.&nbsp; Equilibrium simulations

First, we run a short MD simulation with position restraints to equilibrate the solvent and ions around the protein/ligand. Then, we run tha actual production simulation.

In [75]:
#@title 3.1&nbsp; Prepare TPR files for short equilibration

tpr_files = fe.prepare_simulation(simType='eq_posre', prevSim='em_posre')

-----------------------------------------
Preparing simulation: eq_posre
-----------------------------------------


In [86]:
tpr_files[0]

'/content/pmxrun_1667587093/lysopath/lysozyme_benzene/water/stateA/run1/eq_posre/tpr.tpr'

In [89]:
! ls /content/pmxrun_1667587093/lysopath/lysozyme_benzene/water/stateA/run1/eq_posre

In [ ]:
# Read the TPR files and run all minimizations.
for tpr_file in tpr_files:

  # If minimization has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Load TPR file.
  input_tpr = gmxapi.read_tpr(tpr_file)

  # Specify command line flags and arguments to be passed to mdrun.
  # Note: energy minimization cannot run PME on GPU.
  deffnm = tpr_file.replace('tpr.tpr', 'md')
  md = gmxapi.mdrun(input_tpr, runtime_args={
      '-nb': 'gpu', '-deffnm': deffnm, '-o': deffnm})

  # Run the simulation.
  md.run()

In [ ]:
#@title 3.2&nbsp; Run short equilibration

In [ ]:
#@title 3.3&nbsp; Prepare TPR files for equilibrium simulations

In [ ]:
#@title 3.4&nbsp; Run equilibrium simulations

## 4.&nbsp; Non-equilibrium simulations

In [ ]:
#@title 4.1&nbsp; Extract frames from equilibrium runs

In [ ]:
#@title 4.2&nbsp; Prepare TPR files

In [90]:
#@title 4.3&nbsp; Run non-equilibrium simulations